<a href="https://colab.research.google.com/github/anngook/farmers-note-ml-dev/blob/develop/disease_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# 1. 라이브러리 설치 및 임포트
# ==============================================================================

!pip install efficientnet_pytorch
!pip install torchmetrics
!pip install kagglehub

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset # ConcatDataset 사용
from torchvision import datasets, transforms
from efficientnet_pytorch import EfficientNet
import os
import time
import copy
from google.colab import drive # 모델 저장을 위해 필요
# ⚠️ matplotlib.pyplot 제거

# GPU 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

# 하이퍼파라미터
BATCH_SIZE = 16
NUM_EPOCHS = 10
LEARNING_RATE = 0.001
MODEL_NAME = 'efficientnet-b0'
INPUT_SIZE = EfficientNet.get_image_size(MODEL_NAME)


print("다운로드된 데이터셋 경로:", path)

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=f69067074898ad0a455dea8f6cc3e0924a6394da680af5c309e23ddc7a52bd29
  Stored in directory: /root/.cache/pip/wheels/9c/3f/43/e6271c7026fe08c185da2be23c98c8e87477d3db63f41f32ad
Successfully built efficientnet_pytorch
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 15.6 MB/s eta 0:00:00


100%|██████████| 8.03G/8.03G [01:33<00:00, 91.9MB/s]

Extracting files...


다운로드된 데이터셋 경로: /root/.cache/kagglehub/datasets/loki4514/rice-leaf-diseases-detection/versions/8


In [ ]:
# GPU 사용 설정 (⚠️ 이 부분이 실행되어야 합니다!)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

사용 장치: cuda:0


In [ ]:
DATA_ROOT = '/root/.cache/kagglehub/datasets/loki4514/rice-leaf-diseases-detection/versions/8'

print("루트 경로의 내부 목록을 확인합니다:")
print("==================================")
# 'ls -F' 명령어로 DATA_ROOT 내부 목록을 출력합니다.
!ls -F {DATA_ROOT}

루트 경로의 내부 목록을 확인합니다:
Rice_Leaf_AUG/	Rice_Leaf_Diease/


In [ ]:
# ==============================================================================
# 2. 데이터셋 경로 설정
# ==============================================================================
# 모든 학습 및 검증 경로를 리스트에 통합

path_rice = kagglehub.dataset_download("loki4514/rice-leaf-diseases-detection")
RICE_TRAIN_DIR = os.path.join(path_rice, 'Rice_Leaf_Diease', 'Rice_Leaf_Diease', 'train')
RICE_VAL_DIR = os.path.join(path_rice, 'Rice_Leaf_Diease', 'Rice_Leaf_Diease', 'test')


ALL_TRAIN_DIRS = [RICE_TRAIN_DIR, STRAWBERRY_TRAIN_DIR, POTATO_TRAIN_DIR]
ALL_TEST_DIRS = [RICE_VAL_DIR, STRAWBERRY_VAL_DIR, POTATO_VAL_DIR]


print(f"최종 TRAIN 경로: {TRAIN_DIR}")
print(f"최종 VAL 경로: {VAL_DIR}")

최종 TRAIN 경로: /root/.cache/kagglehub/datasets/loki4514/rice-leaf-diseases-detection/versions/8/Rice_Leaf_Diease/Rice_Leaf_Diease/train
최종 VAL 경로: /root/.cache/kagglehub/datasets/loki4514/rice-leaf-diseases-detection/versions/8/Rice_Leaf_Diease/Rice_Leaf_Diease/test


In [ ]:
# ==============================================================================
# 3. 데이터 로드 및 ConcatDataset 통합
# ==============================================================================

# 데이터 변환 (모든 작물에 동일하게 적용)
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(INPUT_SIZE), transforms.RandomResizedCrop(INPUT_SIZE),
        transforms.RandomHorizontalFlip(), transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    'test': transforms.Compose([
        transforms.Resize(INPUT_SIZE), transforms.CenterCrop(INPUT_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

train_datasets = []
test_datasets = []
all_class_names = []

# 학습 데이터 통합
for dir_path in ALL_TRAIN_DIRS:
    if os.path.exists(dir_path):
        current_dataset = datasets.ImageFolder(dir_path, data_transforms['train'])
        train_datasets.append(current_dataset)
        all_class_names.extend(current_dataset.classes)
        print(f"✅ {os.path.basename(dir_path)} 로드 완료. 클래스: {current_dataset.classes}")
    else:
        print(f"❌ 경로를 찾을 수 없음 (Train): {dir_path}. 이 데이터셋은 제외됩니다.")

# 검증 데이터 통합
for dir_path in ALL_TEST_DIRS:
    if os.path.exists(dir_path):
        current_dataset = datasets.ImageFolder(dir_path, data_transforms['test'])
        test_datasets.append(current_dataset)
    # ⚠️ 검증 데이터의 클래스 이름은 학습 데이터와 동일하다고 가정합니다.

# 💡 모든 작물 데이터셋을 하나의 큰 데이터셋으로 합치기
combined_train_dataset = ConcatDataset(train_datasets)
combined_test_dataset = ConcatDataset(test_datasets)

# 최종 클래스 목록 (중복 제거)
final_class_names = sorted(list(set(all_class_names)))
NUM_CLASSES = len(final_class_names)

dataloaders = {
    'train': DataLoader(combined_train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4),
    'test': DataLoader(combined_test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
}

dataset_sizes = {'train': len(combined_train_dataset), 'test': len(combined_test_dataset)}

print(f"\n✅ 통합 학습 데이터셋 크기: {dataset_sizes['train']}")
print(f"✅ 통합 검증 데이터셋 크기: {dataset_sizes['test']}")
print(f"✅ 통합된 전체 클래스 개수: {NUM_CLASSES}")

In [ ]:
# ==============================================================================
# 4. 모델 설정 (EfficientNet-B0 전이 학습)
# ==============================================================================

model_ft = EfficientNet.from_pretrained(MODEL_NAME, num_classes=NUM_CLASSES).to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model_ft.parameters(), lr=LEARNING_RATE)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth


100%|██████████| 20.4M/20.4M [00:00<00:00, 205MB/s]


Loaded pretrained weights for efficientnet-b0


In [ ]:
# ==============================================================================
# 5. 모델 학습 함수 (test 키 사용으로 수정)
# ==============================================================================

def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}'); print('-' * 10)

        # ⚠️ 'val' 대신 'test' 키를 사용합니다.
        for phase in ['train', 'test']:
            if phase == 'train': model.train()
            else: model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train': scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            # ⚠️ 출력 메시지도 'test'로 수정
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # ⚠️ 'test' 단계의 정확도가 최고 기록일 때 모델을 저장합니다.
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print(f'-> New best model found! Saving weights with Acc: {best_acc:.4f}')

    time_elapsed = time.time() - since
    print(f'\n학습 완료! 총 시간: {time_elapsed // 60:.0f}분 {time_elapsed % 60:.0f}초')
    print(f'최고 검증 정확도: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# ==============================================================================
# 6. 모델 학습 실행 및 저장
# ==============================================================================

# 모델 학습 시작
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=NUM_EPOCHS)


In [ ]:
# 현재 할당된 GPU 정보 확인
!nvidia-smi

Wed Oct  8 06:47:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             33W /   70W |    3494MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# 구글 드라이브 마운트 (저장을 위해 필요합니다)
drive.mount('/content/drive')
MODEL_SAVE_PATH = '/content/drive/MyDrive/efficientnet_rice_disease_best_model.pth'

# 모델 저장
torch.save(model_ft.state_dict(), MODEL_SAVE_PATH)
print(f"\n✅ 모델 가중치가 {MODEL_SAVE_PATH}에 저장되었습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

✅ 모델 가중치가 /content/drive/MyDrive/efficientnet_rice_disease_best_model.pth에 저장되었습니다.
